In [1]:
using GamsStructure
using JuMP
using Ipopt
using Complementarity

1. Define sets and 2. Read data

In [7]:
GU = GamsUniverse()

@create_set!(GU,:s,"Load Segments (peak to base load)",begin
    s1,""
    s2,""
    s3,""
    s4,""
    s5,""
    s6,""
    s7,""
    s8,""
    s9,""
end)

@create_set!(GU,:j,"Generating Units",begin
    Nuclear,""
    Coal,""
    Gas,""
    Diesel,""
end)

@create_set!(GU,:i,"Demand Categories",begin
    rsd, "Residential"
    com, "Commercial"
    ind, "Industrial"
end)

@create_set!(GU,:load_domain,"Domain of load",begin
    qref,""
    hours,""
    rsd,""
    com,""
    ind,""
end)

@create_set!(GU,:supply_domain,"Domain of supply",begin
    mc,""
    cap,""
end);


@load_parameters!(GU,"qp_data",begin
    :load, (:s,:load_domain), description => "Electricity load"
    :supply, (:j,:supply_domain), description => "Electricity supply technology"
end)

@create_parameters(GU,begin
    :mc,  (:j,),	"Marginal cost of dispatch"
    :cap, (:j,),	"Capacity constraint"
    :qref,(:s,),	"Reference demand"
    :h,   (:s,),	"Hours"
    :pref, (:s,),   "Refernce price"
    :dref, (:i,:s), "Reference demand"
    :epsilon, (:i,), "Elasticity of demand"
    :Y_marginal, (:j,:s), "Marginal values in the refence step"
end)


GU[:epsilon][:i] = [.1,.2,.5];


In [8]:
GU[:h][:s] = GU[:load][:s,[:hours]]
GU[:mc][:j] = GU[:supply][:j,[:mc]]
GU[:cap][:j] = GU[:supply][:j,[:cap]]
GU[:qref][:s] = GU[:load][:s,[:qref]];

3. Compute reference prices for each of the segments using 
a linear program.

In [9]:
function reference_prices(GU)

    J = [j for j∈GU[:j]]
    S = [s for s∈GU[:s]]

    m = Model(Ipopt.Optimizer)



    @variables(m,begin
        Y[J,S]>=0
    end)


    for s∈S
        set_upper_bound.(Y[J,s],GU[:supply][J,[:cap]])
    end

    @objective(m,Min, sum(GU[:mc][[j]]*Y[j,s] for s∈S,j∈J))

    @constraint(m,demand[s=S],
        sum(Y[j,s] for j∈J) == GU[:qref][[s]]
    )

    return m
end

reference_prices (generic function with 1 method)

In [10]:
ref = reference_prices(GU)

set_silent(ref)

optimize!(ref)


#Loops probably aren't needed here, but I'm in a hurry :\
for s∈GU[:s]
    GU[:pref][[s]] = dual(ref[:demand][s])
end

for i∈GU[:i],s∈GU[:s]
    GU[:dref][[i],[s]] = GU[:qref][[s]] * GU[:load][[s],[i]]
end

for j∈GU[:j],s∈GU[:s]
    GU[:Y_marginal][[j],[s]] = dual.(UpperBoundRef.(ref[:Y][j,s]))
end



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************



4. Set up a calibrated equilibrium model:

In [11]:
function calibrated_model_mcp(GU)
    I = [i for i∈GU[:i]]
    J = [j for j∈GU[:j]]
    S = [s for s∈GU[:s]]

    dref = GU[:dref]
    epsilon = GU[:epsilon]
    mc = GU[:mc]
    cap = GU[:cap]
    pref = GU[:pref]


    m = MCPModel()

    @variables(m,begin
        Y[J,S]>=0
        D[i=I,s=S]>=0, (start = dref[[i],[s]],)
        PI[j=J,s=S]>=0, (start = -GU[:Y_marginal][[j],[s]],)
        P[s=S]>=0, (start = pref[[s]],)
    end)

    @mapping(m,aggdemand[i=I,s=S],
        D[i,s] - dref[[i],[s]] * (1 - epsilon[[i]] * (P[s]/pref[[s]]-1))
    )

    @mapping(m,supplydemand[s=S],
        sum(Y[j,s] for j∈J) - sum(D[i,s] for i∈I)
    )

    @mapping(m,profit[j=J,s=S],
        mc[[j]] + PI[j,s] - P[s] 
    )

    @mapping(m,capacity[j=J,s=S],
        cap[[j]] - Y[j,s]
    )   


    @complementarity(m,aggdemand,D)
    @complementarity(m,supplydemand,P)
    @complementarity(m,profit,Y)
    @complementarity(m,capacity,PI)

    return m

end

calibrated_model_mcp (generic function with 1 method)

In [12]:
cal_mcp = calibrated_model_mcp(GU)

solveMCP(cal_mcp,cumulative_iteration_limit=1)

print(generate_report(cal_mcp))


Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_E20F.tmp
 > cumulative_iteration_limit 1
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             1.9614e+00             0.0e+00 (supplydemand[s1])
    1    12    19    52 1.9614e+00  5.4e-05    2.0e-02 (supplydemand[s1])
    2    13    25    71 4.8285e-01  1.0e+00    1.8e-02 (profit[Coal,s4])
    3    14     0    65 1.4518e-02  1.0e+00    4.8e-03 (capacity[Nuclear,s4])
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0    15     4 1.4518e-02           I 1.5e-04 6.6e-03 (capacity[Nuclea)
    1     1    16     5 1.4790e-06  1.3e-311 SU 5.8e-05 1.1e-06 (profit[Diesel,s)

Major Iterations. . . . 1
Minor Iterations. . . . 1
Restarts. . . . . . . . 0
Crash Ite

var_name	 value		 margin
D[rsd,s1]		 0.7		 0.0
D[com,s1]		 0.2		 0.0
D[ind,s1]		 0.1		 0.0
D[rsd,s2]		 0.395		 0.0
D[com,s2]		 0.158		 0.0
D[ind,s2]		 0.237		 0.0
D[rsd,s3]		 0.236		 0.0
D[com,s3]		 0.177		 0.0
D[ind,s3]		 0.177		 0.0
D[rsd,s4]		 0.2		 0.0
D[com,s4]		 0.15		 0.0
D[ind,s4]		 0.15		 0.0
D[rsd,s5]		 0.132		 -0.0
D[com,s5]		 0.132		 -0.0
D[ind,s5]		 0.176		 -0.0
D[rsd,s6]		 0.08		 -0.0
D[com,s6]		 0.12		 -0.0
D[ind,s6]		 0.2		 -0.0
D[rsd,s7]		 0.035		 -0.0
D[com,s7]		 0.105		 -0.0
D[ind,s7]		 0.21		 -0.0
D[rsd,s8]		 0.028		 0.0
D[com,s8]		 0.084		 0.0
D[ind,s8]		 0.168		 0.0
D[rsd,s9]		 0.023		 0.0
D[com,s9]		 0.069		 0.0
D[ind,s9]		 0.138		 0.0
P[s1]		 9.0		 -0.0
P[s2]		 7.0		 -0.0
P[s3]		 6.0		 -0.0
P[s4]		 6.0		 -0.0
P[s5]		 6.0		 0.0
P[s6]		 6.0		 -0.0
P[s7]		 6.0		 -0.0
P[s8]		 4.0		 -0.0
P[s9]		 4.0		 -0.0
Y[Nuclear,s1]		 0.3		 0.0
Y[Coal,s1]		 0.3		 0.0
Y[Gas,s1]		 0.2		 0.0
Y[Diesel,s1]		 0.2		 -0.0
Y[Nuclear,s2]		 0.3		 0.0
Y[Coal,s2]		 0.3		 0.0
Y[Gas,s2]		 0.19	

5. Set up a nonlinear programming model which represents the
equilibrium allocation

6. Solve a counterfactual simulation:


In [13]:
function equilibrium_allocation_nlp(GU)
    I = [i for i∈GU[:i]]
    J = [j for j∈GU[:j]]
    S = [s for s∈GU[:s]]

    dref = GU[:dref]
    epsilon = GU[:epsilon]
    mc = GU[:mc]
    cap = GU[:cap]
    pref = GU[:pref]

    m = Model(Ipopt.Optimizer)

    @variables(m,begin
        Y[J,S]>=0
        D[i=I,s=S]>=0, (start = dref[[i],[s]],)
        PI[j=J,s=S]>=0, (start = -GU[:Y_marginal][[j],[s]],)
        P[s=S]>=0, (start = pref[[s]],)
        K[J]>=0
    end)

    
    @objective(m,Max, 
        sum(pref[[s]]*D[i,s] * (1 + 1/epsilon[[i]] * (1-D[i,s]/(2*dref[[i],[s]]))) for i∈I,s∈S)
        - sum(mc[[j]]*Y[j,s] for j∈J,s∈S)
    )


    @constraint(m,supplydemand[s=S],
        sum(Y[j,s] for j∈J) == sum(D[i,s] for i∈I)
    )

    @constraint(m,profit[j=J,s=S],
        mc[[j]] + PI[j,s] >= P[s] 
    )

    @constraint(m,capacity[j=J,s=S],
        cap[[j]] >= Y[j,s]
    )


    return m

end

equilibrium_allocation_nlp (generic function with 1 method)

In [14]:
GU[:mc][[:Coal]]  = 6

benchmark = equilibrium_allocation_nlp(GU)
set_silent(benchmark)
optimize!(benchmark)

GU[:mc][[:Coal]]  = 2*GU[:mc][[:Coal]]

shock = equilibrium_allocation_nlp(GU)
set_silent(shock)
optimize!(shock)

out = "Cumulative Load Segments\n"
x = [0]
for s∈GU[:s]
    push!(x, x[end]+GU[:load][[s],[:hours]])
    out*="$s -> $(x[end])\n"
end
print(out)

out = "\t\tBenchmark\tCounterfactual\n"
for s∈GU[:s]
    b = round(sum(value.(benchmark[:D][:,s])),digits=4)
    c = round(sum(value.(shock[:D][:,s])),digits=4)
    out *= "$s\t\t$b\t\t$c\n"
end
print(out)


Cumulative Load Segments
s1 -> 310
s2 -> 1060
s3 -> 2080
s4 -> 4540
s5 -> 6450
s6 -> 7030
s7 -> 7610
s8 -> 8210
s9 -> 8750


		Benchmark	Counterfactual
s1		1.0		0.9467
s2		0.79		0.7358
s3		0.59		0.5163
s4		0.5		0.4792
s5		0.44		0.4187
s6		0.4		0.378
s7		0.35		0.3284
s8		0.28		0.28
s9		0.23		0.23


The plot command in GAMS doesn't run on my machine ¯\\_(ツ)_/¯